In [1]:
import re #Regular Expression package
import pandas as pd #Pandas dataframes and etc.
import os #Used for shell commands
import numpy as np #NUmpy package
import matplotlib.pyplot as plt #Matplotlib graphing package
%matplotlib inline
import nltk #NLTK NLP Package
from nltk.tokenize import word_tokenize #NLTK NLP Package
np.random.seed(0)
from nltk.corpus import gutenberg, stopwords #NLTK NLP Package
from nltk.collocations import * #NLTK NLP Package
from nltk import FreqDist #NLTK NLP Package
from nltk import word_tokenize #NLTK NLP Package
from nltk.stem.porter import * #NLTK Stemming
from nltk.stem import WordNetLemmatizer #NLTK Lemmatization
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator #WordCloud Package
# from PIL import Image ##WordCloud Package assistance for WSJ image

Now that we have the data for both Fox News and CNN media websites we need to clean the text data. The dataframes are organized in the same format as you can see below.   

Columns are as follows:  
    -Date  
    -Subheadings

In [2]:
FOX_2020= pd.read_csv('FOX_2020', usecols=[1,2])
FOX_2020.head()

,Date,Text
0,20200101,'The Five' make their predictions for 2020
1,20200101,The best of 'The Five' in 2019
2,20200101,Hate crimes at houses of worship up nearly 35 ...
3,20200101,US sending 750 troops to Middle East
4,20200101,Pompeo postpones trip to Ukraine after attack ...


In [3]:
FOX_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34539 entries, 0 to 34538
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    34539 non-null  int64 
 1   Text    34537 non-null  object
dtypes: int64(1), object(1)
memory usage: 539.8+ KB


In [4]:
FOX_2020.isna().sum()

Date    0
Text    2
dtype: int64

In [5]:
NaN_FOX= FOX_2020.isnull()

In [6]:
NaN= NaN_FOX.any(axis=1)

In [7]:
FOX_2020[NaN]

,Date,Text
13736,20200306,NaN
13750,20200306,NaN


Since it is just two values we are going to just drop these two values.

In [8]:
FOX_2020.dropna(inplace=True)

In [9]:
FOX_2020.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 34537 entries, 0 to 34538
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    34537 non-null  int64 
 1   Text    34537 non-null  object
dtypes: int64(1), object(1)
memory usage: 809.5+ KB


In [10]:
FOX_2020.isna().sum()

Date    0
Text    0
dtype: int64

In [11]:
stopwords = stopwords.words('english')
stopwords+= list(string.punctuation)
stopwords+=["’", '‘','said','says', "'s",'”','“',"n't",'-', "''", 'get', '--', '–']

In [12]:
stopwords+= list(string.punctuation)

In [13]:
stopwords_lowercase=['i','me','my','myself','we','our','ours','ourselves',
 'you', "you're", "you've", "you'll","you'd", 'your','yours', 'yourself',
'yourselves', 'he','him','his', 'himself','she', "she's",
 'her', 'hers','herself','it', "it's",'its', 'itself','they', 'them','their', 'theirs',
 'themselves','what', 'which','who', 'whom','this', 'that',
 "that'll", 'these','those', 'am','is', 'are','was', 'were',
 'be', 'been','being', 'have', 'has', 'had','having', 'do',
 'does', 'did','doing', 'a', 'an', 'the','and', 'but',
 'if', 'or','because','as','until', 'while','of','at','by', 'for','with','about','against',
 'between','into','through','during', 'before','after','above','below',
 'to','from','up','down', 'in','out','on','off',
 'over','under','again','further', 'then','once','here','there',
 'when','where','why','how', 'all','any','both','each', 'few','more',
 'most','other', 'some','such', 'no','nor', 'not','only',
 'own','same', 'so','than', 'too','very', 's','t',
 'can','will','just','don', "don't",'should', "should've",'now',
 'd','ll', 'm','o', 're','ve', 'y','ain',
 'aren',"aren't", 'couldn',"couldn't", 'didn',"didn't", 'doesn',"doesn't",
 'hadn',"hadn't", 'hasn',"hasn't", 'haven',"haven't", 'isn',"isn't",
 'ma','mightn', "mightn't",'mustn', "mustn't",'needn', "needn't",'shan',
 "shan't",'shouldn',"shouldn't",'wasn',"wasn't",
 'weren',"weren't", 'won',"won't", 'wouldn',"wouldn't"]

In [14]:
for x in stopwords_lowercase:
    stopwords.append(x.capitalize())

In [15]:
def freq_dist_tokenizer(df):
    tokenized_df= df.Text.apply(nltk.word_tokenize)
    tokens=[]
    for x in tokenized_df:
        for y in x:
            if y not in stopwords:
                tokens.append(y.lower())
    freq_dist=FreqDist(tokens).most_common(25)
    return freq_dist

In [16]:
tokenized_FOX= FOX_2020.Text.apply(nltk.word_tokenize)

In [17]:
tokenized_FOX

0        ['The, Five, ', make, their, predictions, for,...
1                 [The, best, of, 'The, Five, ', in, 2019]
2        [Hate, crimes, at, houses, of, worship, up, ne...
3             [US, sending, 750, troops, to, Middle, East]
4        [Pompeo, postpones, trip, to, Ukraine, after, ...
                               ...                        
34534    [Seattle, Mayor, Durkan, asks, City, Council, ...
34535    [Georgia, judge, grants, $, 500G, bond, to, At...
34536    [Chad, Daybell, predicted, wife, 's, death, th...
34537    [LA, City, Council, member, announces, '1st, s...
34538    [Florida, woman, had, hitman, kill, stepfather...
Name: Text, Length: 34537, dtype: object

In [18]:
tokens=[]
for x in tokenized_FOX:
    for y in x:
        if y not in stopwords:
            tokens.append(y.lower())

In [19]:
len(tokens)

305290

In [20]:
fox_top_25=FreqDist(tokens).most_common(25)

In [21]:
fox_top_25

[('coronavirus', 6905),
 ('trump', 2762),
 ('new', 1945),
 ('amid', 1349),
 ('police', 1206),
 ('us', 1195),
 ('pandemic', 950),
 ('man', 945),
 ('biden', 819),
 ('report', 793),
 ('say', 762),
 ('woman', 729),
 ('day', 703),
 ('2020', 654),
 ('home', 648),
 ('covid-19', 618),
 ('california', 603),
 ('house', 566),
 ('people', 557),
 ('help', 551),
 ('family', 550),
 ('state', 547),
 ('video', 542),
 ('outbreak', 542),
 ('first', 533)]

1. Create time series modeling of the most popular word throughout each of the months/days
2. Dive into the coronavirus months
3. Break down each months most popular words

# CNN

I discovered that CNN had a lot of missing data so I had to go back and re-webscrape some of the data. I decided to cut that portion out of the jupyter notebook for cleaniness and structural flow.

In [22]:
CNN= pd.read_csv('CNN_2020_Final.csv')

In [23]:
CNN.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20284 entries, 0 to 20283
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Date    20284 non-null  int64 
 1   Text    20239 non-null  object
dtypes: int64(1), object(1)
memory usage: 317.1+ KB


In [24]:
CNN.head()

,Date,Text
0,20200101,2020 new laws
1,20200101,Pope
2,20200101,Zoo fire
3,20200101,Stacy London
4,20200101,Richest person


In [25]:
CNN.isna().sum()

Date     0
Text    45
dtype: int64

In [26]:
CNN.dropna(inplace=True)

In [27]:
CNN.isna().sum()

Date    0
Text    0
dtype: int64

In [28]:
freq_dist_tokenizer(CNN)

[('trump', 1684),
 ('coronavirus', 1373),
 ('new', 904),
 ('us', 772),
 ('covid-19', 500),
 ('could', 455),
 ('world', 380),
 ('see', 371),
 ('opinion', 349),
 ('pandemic', 348),
 ('analysis', 347),
 ('2020', 326),
 ('may', 320),
 ('people', 314),
 ('one', 298),
 ('day', 288),
 ('home', 285),
 ('first', 272),
 ('man', 271),
 ('biden', 270),
 ('virus', 266),
 ('house', 263),
 ('police', 259),
 ('back', 247),
 ('help', 241)]

In [29]:
fox_top_25

[('coronavirus', 6905),
 ('trump', 2762),
 ('new', 1945),
 ('amid', 1349),
 ('police', 1206),
 ('us', 1195),
 ('pandemic', 950),
 ('man', 945),
 ('biden', 819),
 ('report', 793),
 ('say', 762),
 ('woman', 729),
 ('day', 703),
 ('2020', 654),
 ('home', 648),
 ('covid-19', 618),
 ('california', 603),
 ('house', 566),
 ('people', 557),
 ('help', 551),
 ('family', 550),
 ('state', 547),
 ('video', 542),
 ('outbreak', 542),
 ('first', 533)]

In [30]:
counter=1
for x in fox_top_25:
    print(str(counter) + '. ' + x[0]+ ' (' + str(x[1]) + ')')
    counter+=1

1. coronavirus (6905)
2. trump (2762)
3. new (1945)
4. amid (1349)
5. police (1206)
6. us (1195)
7. pandemic (950)
8. man (945)
9. biden (819)
10. report (793)
11. say (762)
12. woman (729)
13. day (703)
14. 2020 (654)
15. home (648)
16. covid-19 (618)
17. california (603)
18. house (566)
19. people (557)
20. help (551)
21. family (550)
22. state (547)
23. video (542)
24. outbreak (542)
25. first (533)


In [31]:
CNN_Top_25=[('trump', 1684),
 ('coronavirus', 1373),
 ('new', 904),
 ('us', 772),
 ('covid-19', 500),
 ('could', 455),
 ('world', 380),
 ('see', 371),
 ('opinion', 349),
 ('pandemic', 348),
 ('analysis', 347),
 ('2020', 326),
 ('may', 320),
 ('people', 314),
 ('one', 298),
 ('day', 288),
 ('home', 285),
 ('first', 272),
 ('man', 271),
 ('biden', 270),
 ('virus', 266),
 ('house', 263),
 ('police', 259),
 ('back', 247),
 ('help', 241)]

In [32]:
counter=1
for x in CNN_Top_25:
    print(str(counter) + '. ' + x[0]+ ' (' + str(x[1]) + ')')
    counter+=1

1. trump (1684)
2. coronavirus (1373)
3. new (904)
4. us (772)
5. covid-19 (500)
6. could (455)
7. world (380)
8. see (371)
9. opinion (349)
10. pandemic (348)
11. analysis (347)
12. 2020 (326)
13. may (320)
14. people (314)
15. one (298)
16. day (288)
17. home (285)
18. first (272)
19. man (271)
20. biden (270)
21. virus (266)
22. house (263)
23. police (259)
24. back (247)
25. help (241)


Preliminary analysis of the 2020 data:                                         
    FoxNews Top 25___________________________ CNN Top 25  
    -Fox News Top 25                           -CNN Top 25
    1. coronavirus (6905)                    1. trump (1684)  
    2. trump (2762)                          2. coronavirus (1373)
    3. new (1945)                            3. new (904)
    4. amid (1349)                           4. us (772)
    5. police (1206)                         5. covid-19 (500)
    6. us (1195)                             6. could (455)
    7. pandemic (950)                        7. world (380)
    8. man (945)                             8. see (371)
    9. biden (819)                           9. opinion (349)
    10. report (793)                        10. pandemic (348)
    11. say (762)                           11. analysis (347)
    12. woman (729)                         12. 2020 (326)
    13. day (703)                           13. may (320)
    14. 2020 (654)                          14. people (314)
    15. home (648)                          15. one (298)
    16. covid-19 (618)                      16. day (288)
    17. california (603)                    17. home (285)
    18. house (566)                         18. first (272)
    19. people (557)                        19. man (271)
    20. help (551)                          20. biden (270)
    21. family (550)                        21. virus (266)
    22. state (547)                         22. house (263)
    23. video (542)                         23. police (259)
    24. outbreak (542)                      24. back (247)
    25. first (533)                         25. help (241)